### Search Engine with tools and agent

In [14]:
# Load env data
import os
from dotenv import load_dotenv
load_dotenv()

# Access HuggingFace API Key
os.environ["HF_TOKEN"] = os.getenv("HF_TOKEN")

In [27]:
# Read langsmith data
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader = WebBaseLoader("https://docs.smith.langchain.com/")
docs = loader.load()

splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
final_documents = splitter.split_documents(docs)
final_documents

[Document(metadata={'source': 'https://docs.smith.langchain.com/', 'title': 'Get started with LangSmith | 🦜️🛠️ LangSmith', 'description': 'LangSmith is a platform for building production-grade LLM applications.', 'language': 'en'}, page_content='Get started with LangSmith | 🦜️🛠️ LangSmith\n\n\n\n\n\n\nSkip to main contentLearn the essentials of LangSmith in the new Introduction to LangSmith course!  Enroll for free. API ReferenceRESTPythonSearchRegionUSEUGo to AppQuick StartObservabilityEvaluationPrompt EngineeringDeployment (LangGraph Platform)AdministrationSelf-hostingPricingReferenceCloud architecture and scalabilityAuthz and AuthnAuthentication methodsdata_formatsEvaluationDataset transformationsRegions FAQsdk_referenceQuick StartOn this pageGet started with LangSmith\nLangSmith is a platform for building production-grade LLM applications.\nIt allows you to closely monitor and evaluate your application, so you can ship quickly and with confidence.\nWith LangSmith you can:'),
 Docum

In [28]:
# Create vector embeddings
from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
embeddings

HuggingFaceEmbeddings(model_name='all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [29]:
# Vector store - FAISS
from langchain_community.vectorstores import FAISS

vectorstore = FAISS.from_documents(documents=final_documents, embedding=embeddings)
retriever = vectorstore.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x7fc7a17e10a0>, search_kwargs={})

In [30]:
# Wikipedia
from langchain.tools import ArxivQueryRun, WikipediaQueryRun
from langchain.utilities import ArxivAPIWrapper, WikipediaAPIWrapper


wikipedia_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=250)
wikipedia = WikipediaQueryRun(api_wrapper=wikipedia_wrapper)
wikipedia.name

'wikipedia'

In [31]:
# Arxiv
from langchain.tools import ArxivQueryRun
from langchain.utilities import ArxivAPIWrapper

arxiv_wrapper = ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=250)
arxiv = ArxivQueryRun(api_wrapper=arxiv_wrapper)
arxiv.name

'arxiv'

In [32]:
from langchain.tools.retriever import create_retriever_tool

retriever_tool = create_retriever_tool(retriever=retriever, name="langsmith-search", description="Search any information about langsimth")
retriever_tool.name

'langsmith-search'

In [33]:
# Create tool with Wikipedia and Arxiv
tools = [wikipedia, arxiv, retriever_tool]
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from '/home/laluprasadmahato@ADCNST.COM/Desktop/Projects/UdemyGenerativeAI/JanGenEnv/lib/python3.12/site-packages/wikipedia/__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=250)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=250)),
 Tool(name='langsmith-search', description='Search any information about langsimth', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x7fc8c537a840>, retriever=VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectors

In [41]:
# Create model
from langchain_groq import ChatGroq

groq_api_key = os.getenv("GROQ_API_KEY")

model = ChatGroq(groq_api_key=groq_api_key, model="llama3-8b-8192")
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x7fc79439b200>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x7fc7949c31a0>, model_name='llama3-8b-8192', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [42]:
# Create prompt from hub
from langchain import hub

prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [43]:
# Create agent
from langchain.agents import create_openai_tools_agent

agent = create_openai_tools_agent(llm=model, tools=tools, prompt=prompt)
agent

RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages

In [44]:
# Create agent executor
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
agent_executor

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag

In [46]:
# Execute agent
agent_executor.invoke({ "input": "What is Generative AI?"})



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'Generative AI'}`


Page: Generative artificial intelligence
Summary: Generative artificial intelligence (generative AI, GenAI, or GAI) is a subset of artificial intelligence that uses generative models to produce text, images, videos, or other forms of data. These mode
Invoking: `arxiv` with `{'query': 'generative AI'}`


Published: 2021-03-29
Title: "Weak AI" is Likely to Never Become "Strong AI", So What is its Greatest Value for us?
Authors: Bin Liu
Summary: AI has surpassed humans across a variety of tasks such as image
classification, playing games (e.g., go, "St
Invoking: `langsmith-search` with `{'query': 'What is the greatest value of weak AI'}`


Learn more about tracing in the observability tutorials, conceptual guide and how-to guides.
5. View your trace​
By default, the trace will be logged to the project with the name default. You should see the following sample output trace logged using the ab

{'input': 'What is Generative AI?',
 'output': "It looks like the tool call yielded a result that is not directly relevant to the user's question about the greatest value of weak AI. The result appears to be a tutorial or guide on using a platform called LangSmith, which is likely related to natural language processing or AI development.\n\nAs the conversation has taken a detour from the original topic, I will respond directly to the user's question.\n\nWhile weak AI may not have the potential to become strong AI, its greatest value lies in its ability to automate and assist humans in various tasks, such as data processing, pattern recognition, and decision-making. Weak AI can also be used to improve the efficiency and accuracy of many industries, including healthcare, finance, and transportation. Additionally, weak AI can help humans make better decisions by providing insights and recommendations based on large datasets and complex algorithms."}

In [47]:
agent_executor.invoke({ "input": "Tell me about Langsmith."})



> Entering new AgentExecutor chain...

Invoking: `langsmith-search` with `{'query': 'Langsmith'}`


Get started with LangSmith | 🦜️🛠️ LangSmith






Skip to main contentLearn the essentials of LangSmith in the new Introduction to LangSmith course!  Enroll for free. API ReferenceRESTPythonSearchRegionUSEUGo to AppQuick StartObservabilityEvaluationPrompt EngineeringDeployment (LangGraph Platform)AdministrationSelf-hostingPricingReferenceCloud architecture and scalabilityAuthz and AuthnAuthentication methodsdata_formatsEvaluationDataset transformationsRegions FAQsdk_referenceQuick StartOn this pageGet started with LangSmith
LangSmith is a platform for building production-grade LLM applications.
It allows you to closely monitor and evaluate your application, so you can ship quickly and with confidence.
With LangSmith you can:

LangSmith + LangChain OSSLangSmith integrates seamlessly with LangChain's open source frameworks langchain and langgraph, with no extra instrumentation needed.If 

{'input': 'Tell me about Langsmith.',
 'output': 'It looks like the tool call "call_6e3n" yielded a large amount of text about LangSmith, a platform for building production-grade LLM applications. LangSmith allows users to closely monitor and evaluate their LLM applications, and provides a range of features such as logging, tracing, and evaluation.\n\nSome of the key points from the text include:\n\n* LangSmith is a standalone platform that can be used with or without LangChain or LangGraph.\n* It provides a range of features, including logging, tracing, and evaluation.\n* LangSmith integrates seamlessly with LangChain\'s open source frameworks.\n* LangSmith is a production-grade platform that allows users to ship LLM applications quickly and with confidence.\n* It provides a range of tutorials and guides to help users get started.\n\nLet me know if you\'d like me to summarize or clarify any of this information!'}

In [49]:
agent_executor.invoke({ "input": "What's the paper 1706.03762 about" })



> Entering new AgentExecutor chain...

Invoking: `arxiv` with `{'query': '1706.03762'}`


Published: 2023-08-02
Title: Attention Is All You Need
Authors: Ashish Vaswani, Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones, Aidan N. Gomez, Lukasz Kaiser, Illia Polosukhin
Summary: The dominant sequence transduction models are based on cThe paper 1706.03762 is titled "Attention Is All You Need" and it was published in 2017. It introduces a new neural network architecture called the Transformer, which is capable of performing tasks such as machine translation, without relying on traditional recurrent neural networks (RNNs) or convolutional neural networks (CNNs). The Transformer uses self-attention mechanisms to allow the model to focus on different parts of the input sequence simultaneously, which allows it to process long-range dependencies more effectively. This paper has had a significant impact on the field of natural language processing and has been widely adopted in many appli

{'input': "What's the paper 1706.03762 about",
 'output': 'The paper 1706.03762 is titled "Attention Is All You Need" and it was published in 2017. It introduces a new neural network architecture called the Transformer, which is capable of performing tasks such as machine translation, without relying on traditional recurrent neural networks (RNNs) or convolutional neural networks (CNNs). The Transformer uses self-attention mechanisms to allow the model to focus on different parts of the input sequence simultaneously, which allows it to process long-range dependencies more effectively. This paper has had a significant impact on the field of natural language processing and has been widely adopted in many applications, including machine translation, text summarization, and question answering.'}